## BCMapHub Service URL Updater - June 7th 2021
### DataBC has been working to improve web service performance and stability for public data access web services available to ArcGIS Online via BCs Map Hub. Historically we have published a single large service with over 700 layers. The single service will be replaced by multiple smaller services published by schema. Duplicate Map Image Layer items will also be deprecated in favour of the corresponding authoritative Feature Layer items. We are now ready to implement the changes.
### https://www2.gov.bc.ca/assets/gov/data/geobc/bcs_map_hub_-_change_notice_and_additional_20210520_1.pdf
#### Michael Dykes (Michael.Dykes@gov.bc.ca)

In [ ]:
# Import required libraries/modules
import json,requests,re
from ipywidgets import *
from difflib import SequenceMatcher
from arcgis.gis import GIS
from arcgis.mapping import WebMap

# Create connection to AGO "home" works for AGO Notebooks, otherwise need to change to something like "gis = GIS('https://governmentofbc.maps.arcgis.com/',username,password)"
gis = GIS("home")

In [ ]:
# Search AGO for Web Maps owned by you, max 500 items (if you need more then change this)
for item in gis.content.search(query="* AND \  owner:" + gis.users.me.username, item_type="Web Map", max_items=500):
    # Open item as Web Map Object
    WebMap_Object = WebMap(item)
    # Iterate through Web Map layers
    WebMapLayerIDs = [x["id"] for x in WebMap_Object.layers]
    for layer in WebMap_Object.layers:
        # Check if you can access URL key from layer JSON (sometimes you can't)
        if 'url' in layer:
            # Check if layer has the old service layer url
            if "mpcm/bcgwpub/MapServer" in layer['url']:
                # Sometimes itemIDs don't exist in a layer JSON, I belive this happens when you have multiple copies of the same feature layer in the same map? Updating the url of one will hopefully update the other?
                if 'itemId' in layer:
                    # Get AGO item from itemID to then go find the updated url
                    Newitem = gis.content.get(layer['itemId'])
                    # Ensure the url has been updated before updating it in your Web Map
                    if "mpcm/bcgwpub/MapServer" not in Newitem.url:
                        # Grab the new url
                        newURL = Newitem.url
                        # Replace the old url with the new in the JSON
                        layer['url'] = newURL
                        # Use update function to finalize the change
                        WebMap_Object.update()
                        print ("Service URL Updated for " + layer['title'] + " Layer in " + item.title + " Web Map")
print("***Done***")               

In [ ]:
# Find any old web service urls that still exist (from copied layers in Web Maps, Web Mapping Applications, etc)
ToUpdate_Dict = {}
for item in gis.content.search(query="* AND \  owner:" + gis.users.me.username, max_items=1000):
    if item.type != "Code Attachment":
        item_data = item.get_data()
        item_data_Str = str(item_data)
        if "mpcm/bcgwpub/MapServer" in item_data_Str:
            urllist = re.findall("'(https://(?:.*?))'", item_data_Str)
            for row in urllist:
                if "mpcm/bcgwpub/MapServer" in row:
                    if (item.title,item.type,item.id) in ToUpdate_Dict:
                        ToUpdate_Dict[(item.title,item.type,item.id)].append(row)
                    else:
                        ToUpdate_Dict[(item.title,item.type,item.id)] = [row]
                        
if ToUpdate_Dict:
    URL_List = []
    for row in ToUpdate_Dict.values():
        for x in row:
            URL_List.append(x)

    UniqueURL_List = list(set(URL_List))
    print("Old Service URLs Found. Continue.")
else:
    print("You got everything! No More 'mpcm/bcgwpub/MapServer' References in your AGO items.")

In [ ]:
# Check Old BCGW Web Service for Titles and Service URLs to Compare to the New BCGW Web Service
RESTServer = r'https://maps.gov.bc.ca/arcgis/rest/services/mpcm/bcgwpub/MapServer/'
response = requests.get(RESTServer + '?f=json')
sitejson = response.json()

UrlDict = {}
for layer in sitejson['layers']:
    REST_serviceURL = RESTServer + str(layer['id'])
    if REST_serviceURL in UniqueURL_List:
        UrlDict[layer['name']] = REST_serviceURL

In [ ]:
# Build Dictionary of New BCGW Web Service URLs and Titles
NewREST_serviceURL = r'https://maps.gov.bc.ca/arcgis/rest/services/whse?f=json'
Newresponse = requests.get(NewREST_serviceURL)
Newsitejson = Newresponse.json()

whse_services_list = []
for row in Newsitejson['services']:
    whse_services_list.append(row['name'])

UrlDict_New = {}
for row in whse_services_list:
    Sub_Resturl = r'https://maps.gov.bc.ca/arcgis/rest/services/' + row + '/MapServer/'
    Sub_response = requests.get(Sub_Resturl + '?f=json')
    Sub_sitejson = Sub_response.json()
    for layer in Sub_sitejson['layers']:
        UrlDict_New[layer['name']] = Sub_Resturl + str(layer['id'])

In [ ]:
# Sort out Suggestions for Updating the URL
Suggestions_Dict = {}
for k,v in UrlDict.items():
    # If the Layer Titles Don't Match Exactly
    if k not in UrlDict_New:
        SimliarityList = []
        StrInList = []
        # Iterate through the New Web Service Layers
        for k2 in UrlDict_New.keys():
            # Check if the Old Title is a Substring of the New Title (Ex. Old is "Otter Habitat", New could be "Otter Habitat - Conservation")
            if k in k2:
                toappend = k,k2
                StrInList.append(toappend)
            # Use Sequence Matcher to get a Ratio of How Similar the Title Strings are.
            Str_Similarity = SequenceMatcher(a=k,b=k2).ratio()
            # Find only Strings that are 80% Similar
            if Str_Similarity >= 0.8:
                toappend2 = k,k2,Str_Similarity
                SimliarityList.append(toappend2)
        
        # Deal with all Substring Matches Found, Add them to Suggestions Dictionary
        if StrInList:
            if len(StrInList) > 1:
                if (k,v) in Suggestions_Dict:
                    Suggestions_Dict[(k,v)].extend(StrInList)
                else:
                    Suggestions_Dict[(k,v)] = StrInList
            else:
                if (k,v) in Suggestions_Dict:
                    Suggestions_Dict[(k,v)].append(StrInList[0])
                else:
                    Suggestions_Dict[(k,v)] = [StrInList[0]]
           
        # Deal with all >80% String Matches Found, Add them to Suggestions Dictionary
        elif SimliarityList:
            if len(SimliarityList) > 1:
                if (k,v) in Suggestions_Dict:
                    Suggestions_Dict[(k,v)].extend(sorted(SimliarityList,key=lambda x : float(x[2]),reverse=True))
                else:
                    Suggestions_Dict[(k,v)] = sorted(SimliarityList,key=lambda x : float(x[2]),reverse=True)
            else:
                if (k,v) in Suggestions_Dict:
                    Suggestions_Dict[(k,v)].append[SimliarityList[0]]
                else:
                    Suggestions_Dict[(k,v)] = [SimliarityList[0]]
                
        # No Match - *Bad            
        else:
            Suggestions_Dict[(k,v)] = "No Match Found" 
            
    # Perfect Match        
    else:
        Suggestions_Dict[(k,v)] = UrlDict_New[k]